Imports

In [1]:
rm -rf training_checkpoints/

SyntaxError: invalid syntax (<ipython-input-1-7cdd8c271778>, line 1)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [96]:
import tensorflow as tf

import numpy as np
import os
import time
"""
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
"""

'\nfrom google.colab import drive\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n'

Dataset

In [97]:
drive.mount('/content/gdrive')

NameError: name 'drive' is not defined

In [98]:
path_to_file = 'lab.txt'

lectura de la informacion

In [99]:
text = open(path_to_file, 'rb').read().decode(encoding='UTF-8')

In [100]:
#visualizar los primeros caracteres
print(text[:250])

years thereafter oil fields platforms named pagan godswe love brownchad awesome kids holding fort work later usual kids busy together playing skylander xbox together kyan cashed piggy bank wanted game bad used gift card birthday saving money get neve


In [101]:
vocab = sorted(set(text))


Vectorización

In [102]:
char2idx = {u:i for i, u in enumerate(vocab)} #mapeo a indices de caracter unico
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [103]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '+' :   1,
  '0' :   2,
  '1' :   3,
  '2' :   4,
  '3' :   5,
  '4' :   6,
  '5' :   7,
  '6' :   8,
  '7' :   9,
  '8' :  10,
  '9' :  11,
  '=' :  12,
  'a' :  13,
  'b' :  14,
  'c' :  15,
  'd' :  16,
  'e' :  17,
  'f' :  18,
  'g' :  19,
  ...
}


In [104]:
seq_length = 100 #tamano del input
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) #creacion de lineas de entrenamiento

In [105]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [106]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [107]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'years thereafter oil fields platforms named pagan godswe love brownchad awesome kids holding fort wo'
Target data: 'ears thereafter oil fields platforms named pagan godswe love brownchad awesome kids holding fort wor'


In [108]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 37 ('y')
  expected output: 17 ('e')
Step    1
  input: 17 ('e')
  expected output: 13 ('a')
Step    2
  input: 13 ('a')
  expected output: 30 ('r')
Step    3
  input: 30 ('r')
  expected output: 31 ('s')
Step    4
  input: 31 ('s')
  expected output: 0 (' ')


In [109]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000 #tamano del buffer
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [110]:
vocab_size = len(vocab) #tamano del vocabulario
embedding_dim = 256 #tamano de insercion
rnn_units = 1024 #unidades rnn

In [111]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size): #modelo a tres capas ocultas
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [112]:
model = build_model(  #build del modelo
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [113]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 659) # (batch_size, sequence_length, vocab_size)


In [114]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           168704    
_________________________________________________________________
gru_5 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_5 (Dense)              (64, None, 659)           675475    
Total params: 4,782,483
Trainable params: 4,782,483
Non-trainable params: 0
_________________________________________________________________


In [115]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

indica en base a vectorización el siguiente caracter

In [116]:
sampled_indices

array([244, 627,  53, 643, 361, 335, 363, 275,  62, 145, 563, 590,  35,
        27, 185, 610,  53, 548, 654, 568, 654, 549, 232, 327, 590, 633,
       389,  63, 114, 317, 382, 356, 393, 513, 597, 322, 213,  53, 123,
       633, 301,  13, 604,  18, 650, 557, 289,  61, 551, 376,  64, 529,
       137, 160, 597, 549, 392, 637, 216, 419, 233,  23, 100, 457,  96,
       108, 233, 242, 644, 310, 275, 376, 449, 334,  18, 404,   7,  69,
       146, 498, 488, 521, 181, 209, 332, 459, 265, 427, 593, 321, 621,
       202,  65, 174, 534,  21, 210, 472, 318, 462], dtype=int64)

Entrenamiento


In [117]:
def loss(labels, logits): #funcion de perdida
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 659)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       6.4895277


Seleeción de optimizador, Adam en este caso

In [118]:
model.compile(optimizer='adam', loss=loss)

Se salvan los checkpoints en el disco.


In [119]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Cantidad de rondas de entrenamiento:




In [120]:
EPOCHS=5

Ejecucion del entrenamiento

In [122]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
2962/2962 [==============================] - 64s 21ms/step - loss: 1.4809
Epoch 2/5
2962/2962 [==============================] - 66s 22ms/step - loss: 1.4646
Epoch 3/5
2962/2962 [==============================] - 66s 22ms/step - loss: 1.4561
Epoch 4/5
2962/2962 [==============================] - 66s 22ms/step - loss: 1.4519
Epoch 5/5
2962/2962 [==============================] - 64s 21ms/step - loss: 1.4501 2s - los


--Control de checkpoints

In [123]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_5'

In [124]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [125]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            168704    
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_6 (Dense)              (1, None, 659)            675475    
Total params: 4,782,483
Trainable params: 4,782,483
Non-trainable params: 0
_________________________________________________________________


--Generación de texto:

In [136]:
def generate_text(model, start_string):
  num_generate = 50 #cantidad de caracteres a generar

  #vectorizacion
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = [] #string para output


  temperature = 0.5 #parametro de aceptacion
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [137]:
print(generate_text(model, start_string=u"everyone")) #generación

everyone mean tell suggestions followed book time dont tel


In [3]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0
